In [298]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import os
from operator import itemgetter
import json

path = './/Data//Emails'
dict_path = './/Data//English Words'
name_path = './/Data//English Names'
additional_path  = './/Data//Additional'

In [3]:
train_data = pd.read_csv(os.path.join(path,'emails.train.csv'))
train_data

,id,text,spam
0,0,Subject: naturally irresistible your corporate...,1
1,2,Subject: unbelievable new homes made easy im ...,1
2,3,Subject: 4 color printing special request add...,1
3,4,"Subject: do not have money , get software cds ...",1
4,5,"Subject: great nnews hello , welcome to medzo...",1
5,6,Subject: here ' s a hot play in motion homela...,1
6,9,Subject: save your money buy getting this thin...,1
7,11,Subject: save your money buy getting this thin...,1
8,12,Subject: brighten those teeth get your teeth...,1
9,14,Subject: fpa notice : ebay misrepresentation o...,1


Possible features: 
* Word frequency
 * Each element of the vector indicates frequency of respective word. Occurances divided by total words
* Length frequency
 * Each element of the vector indicates frequency of word with length i. Occurances dived by total words
* Total number of words
 * Numeric value
* Total number of letters
 * Numeric value
* Number of non-alphabet characeters in subject
* Number of non-alphabet characters in body
* Indicators in subject
 * Reply, forward ..
* Personal greeting
* Presence of personal names
* Number of grammatically incorrect words
* Number of unique words

In [56]:
#Creates dictionary of the form {word(str):number(int)} for a single piece of text
def word_split(text):
    word_dict = {}
    current_word = str()
    for i in range(len(text)):
        if text[i].isalpha():
            current_word += text[i]
            continue
        else:
            if current_word:
                if current_word in word_dict:
                    word_dict[current_word] += 1
                else:
                    word_dict[current_word] = 1
                current_word = str()
            else:
                continue
    return word_dict

#Creates a list of all words in the data set
def local_words(text_list):
    local_word_list = []
    for i in range(len(text_list)):
        text = text_list.iloc[i]
        word_dict = word_split(text)
        for word in word_dict:
            local_word_list.append(word)
    return local_word_list

#Creates a dictionary similar to word_split, but for full data set
def local_word_dict(text_list):
    local_word_dict = {}
    for i in range(len(text_list)):
        word_dict = word_split(text_list.iloc[i])
        local_word_dict = {word: local_word_dict.get(word, 0) + word_dict.get(word, 0) for word in set(local_word_dict)|set(word_dict)}
    return local_word_dict

#Returns the n most frequent words in a dictionary of the type described in word_split
def top_words(n, word_dict):
    assert n < len(word_dict)
    top_dict = {}
    sorted_keys = sorted(word_dict, key=word_dict.get, reverse=True)
    for i in range(n):
        top_dict[sorted_keys[i]] = word_dict[sorted_keys[i]]
    return top_dict
        
#Applying the functions to different subsets of the data            
emails = train_data.text
all_words = local_words(emails)

spam = train_data.text[train_data.spam > 0]
spam_words = local_words(spam)

non_spam = train_data.text[train_data.spam < 1]
non_spam_words = local_words(non_spam)

email_dict = local_word_dict(emails)

spam_dict = local_word_dict(spam)

non_spam_dict = local_word_dict(non_spam)

In [299]:
#Reading external data into files. Dictionary, names, other custom lists

english_dict_file = os.path.join(dict_path, 'words_dictionary.json')
with open(english_dict_file,"r") as english_dictionary:
    english_words_dict = json.load(english_dictionary)
english_words_set = set(english_words_dict.keys())

english_fnames_file = os.path.join(name_path, 'popular-both-first.txt')
first_names = pd.read_csv(english_fnames_file, header=None)
english_fnames_list = first_names.iloc[:, 0]
english_fnames_list = [english_fnames_list[i].lower() for i in range(len(english_fnames_list))
                      if isinstance(english_fnames_list[i], str)]
english_fname_set = set(english_fnames_list)

english_lnames_file = os.path.join(name_path, 'census-dist-2500-last.csv')
last_names = pd.read_csv(english_lnames_file, header=None)
english_lnames_list = last_names.iloc[:, 0]
english_lnames_list = [english_lnames_list[i].lower() for i in range(len(english_lnames_list))
                      if isinstance(english_lnames_list[i], str)]
english_lname_set = set(english_lnames_list)

english_greetings_file = os.path.join(additional_path, 'greetings.txt')
greetings = pd.read_csv(english_greetings_file, header=None)
greetings_list = greetings.iloc[:, 0]
greetings_set = set(greetings_list)

indicator_file = os.path.join(additional_path, 'indicators.txt')
indicators = pd.read_csv(indicator_file, header=None)
indicators_list = indicators.iloc[:, 0]
indicators_set = set(indicators_list)

In [281]:
#Getting proportion of correct words
def text_dict_check(text_dict, english_dict_set):
    correct_words = 0
    incorrect_words = 0
    text_set = set(text_dict.keys())
    for word in text_set:
        if word in english_dict_set:
            correct_words += 1
    return correct_words/len(text_dict)

non_spam_prop = np.mean(np.array([text_dict_check(word_split(non_spam.iloc[i]), english_words_set)
                                  for i in range(len(non_spam))]))
spam_prop = np.mean(np.array([text_dict_check(word_split(spam.iloc[i]), english_words_set)
                              for i in range(len(spam))]))

#Check dictionary for words also in set
def personal_names_check(text_dict, english_name_set):
    personal_names = 0
    text_set = set(text_dict.keys())
    for word in text_set:
        if word in english_name_set:
            personal_names += 1
    return personal_names

In [300]:
#Collecting all features in a numpy array
def create_design_matrix(data, english_dict_set, english_name_set):
    features = 6
    observations = len(data)
    X = np.zeros((observations, features))
    for i in range(observations):
        mail = data.text.iloc[i]
        d = word_split(mail)
        #Proportion of unique words
        X[i, 0] = len(d)/sum(d.values())
        #Proportion correct words
        X[i, 1] = text_dict_check(d, english_words_set)
        #Number of personal names
        X[i, 2] = personal_names_check(d, english_fname_set)
        #Number of last names
        X[i, 3] = personal_names_check(d, english_lname_set)
        #Number of greetings
        X[i, 4] = personal_names_check(d, greetings_set)
        #Number of indicators
        X[i, 5] = personal_names_check(d, indicators_set)
    return X

#Creating numpy array for response
def create_response_vector(data):
    Y = np.array(data.spam)
    Y[Y == 0] = -1
    return Y

In [302]:
from sklearn.linear_model import LogisticRegression

def data_split(data):
    test_indices = np.random.choice(len(data), len(data)//5)
    train_indices = np.array([i for i in range(len(data)) if i not in test_indices])
    Y = create_response_vector(data)
    X = create_design_matrix(data, english_words_set, english_name_set)
    X_train = X[train_indices]
    Y_train = Y[train_indices]
    X_test = X[test_indices]
    Y_test = Y[test_indices]
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = data_split(train_data)
lr = LogisticRegression().fit(X_train, Y_train)
print(lr.score(X_train, Y_train))
print(lr.score(X_test, Y_test))

0.846433990895
0.828358208955


In [321]:
from sklearn.svm import LinearSVC, NuSVC

X_train, X_test, Y_train, Y_test = data_split(train_data)
svm = NuSVC(nu=0.3).fit(X_train, Y_train)
print(svm.score(X_train, Y_train))
print(svm.score(X_test, Y_test))

0.86170212766
0.856965174129


In [354]:
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, Y_train, Y_test = data_split(train_data)
tree = DecisionTreeClassifier(max_depth=8, random_state=0)
tree.fit(X_train, Y_train)
print(tree.score(X_train, Y_train))
print(tree.score(X_test, Y_test))

0.900243309002
0.874378109453


In [342]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, Y_train, Y_test = data_split(train_data)

forest = RandomForestClassifier(n_estimators=10, random_state=0)
forest.fit(X_train, Y_train)
print(forest.score(X_train, Y_train))
print(forest.score(X_test, Y_test))

0.986047922354
0.866915422886
